# Ensemble de Problèmes sur la Tarification des Contrats à Terme

Ce notebook couvre la tarification des contrats à terme pour les actions, les matières premières et les devises, y compris le concept de contrats à terme prépayés.

## 1. Fondements Théoriques

### 1.1 Contrats à Terme

Un **contrat à terme** est un accord pour acheter ou vendre un actif à un moment futur spécifié $T$ pour un prix $F$ (le prix à terme) convenu aujourd'hui.

### 1.2 Formules du Prix à Terme

Toutes les formules ci-dessous supposent une **capitalisation continue**.

#### 1.2.1 Action avec Rendement de Dividende Continu

Pour une action avec un prix au comptant $S$, un rendement de dividende continu $q$ et un taux sans risque $r$ :

$$F_0^T = S_0 \cdot e^{(r-q)T}$$

**Intuition** : Vous gagnez le taux sans risque $r$ mais renoncez au rendement de dividende $q$.

#### 1.2.2 Matière Première avec Taux de Crédit-Bail (Rendement de Commodité)

Pour une matière première avec un prix au comptant $S$, un taux de crédit-bail continu $\delta$ (ou rendement de commodité), et un taux sans risque $r$ :

$$F_0^T = S_0 \cdot e^{(r-\delta)T}$$

**Intuition** : Similaire aux dividendes, le taux de crédit-bail représente l'avantage de détenir la matière première physique.

#### 1.2.3 Devise (Parité des Taux d'Intérêt Couverte)

Pour une paire de devises où $S$ est le taux de change au comptant (domestique par étranger), $r_d$ est le taux sans risque domestique, et $r_f$ est le taux sans risque étranger :

$$F_0^T = S_0 \cdot e^{(r_d - r_f)T}$$

**Important** : La convention du taux de change est importante !
- Si $S$ = USDCAD (prix de 1 USD en CAD), alors :
  - Taux domestique = taux CAD ($r_{CAD}$)
  - Taux étranger = taux USD ($r_{USD}$)
  - $F_{USDCAD} = S_{USDCAD} \cdot e^{(r_{CAD} - r_{USD})T}$

- Si $S$ = CADUSD (prix de 1 CAD en USD), alors :
  - Taux domestique = taux USD ($r_{USD}$)
  - Taux étranger = taux CAD ($r_{CAD}$)
  - $F_{CADUSD} = S_{CADUSD} \cdot e^{(r_{USD} - r_{CAD})T}$

### 1.3 Contrat à Terme Prépayé

Un **contrat à terme prépayé** est la valeur actuelle du prix à terme. Au lieu de payer $F$ au temps $T$, vous payez $F^P$ aujourd'hui.

$$F_0^{P,T} = F_0^T \cdot e^{-rT}$$

En substituant les formules du prix à terme :

- **Action** : $F_0^{P,T} = S_0 \cdot e^{-qT}$
- **Matière première** : $F_0^{P,T} = S_0 \cdot e^{-\delta T}$
- **Devise** : $F_0^{P,T} = S_0 \cdot e^{-r_f T}$

**Remarque** : Pour les devises, le contrat à terme prépayé utilise le taux sans risque **étranger**.

### 1.4 Relations Clés

#### Propriété Réciproque pour les Contrats à Terme sur Devises

Pour les paires de devises :
$$F_{USDCAD} = \frac{1}{F_{CADUSD}}$$

**Cependant, cela ne s'applique PAS aux contrats à terme prépayés !**

$$F^P_{USDCAD} \neq \frac{1}{F^P_{CADUSD}}$$

(sauf si $r_{USD} = r_{CAD}$)

## 2. Ensemble de Problèmes

Nous allons travailler sur les problèmes suivants :

**Problème 1** : Calculer le prix à terme d'une action versant des dividendes

**Problème 2** : Calculer le rendement de dividende à partir du prix à terme

**Problème 3** : Calculer le prix à terme d'une matière première

**Problème 4** : Calculer le taux de crédit-bail à partir du prix à terme

**Problème 5** : Calculer les prix à terme des devises (USDCAD et CADUSD)

**Problème 6** : Vérifier la relation réciproque pour les contrats à terme et les contrats à terme prépayés

## 3. Configuration et Génération de Données

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

In [ ]:
S_stock = np.round(np.random.uniform(80, 150), 0)
r_stock = np.round(np.random.uniform(0.03, 0.08), 3)
q_stock = np.round(np.random.uniform(0.01, 0.04), 3)
T_stock_options = [0.25, 0.5, 0.75, 1.0]
T_stock = np.random.choice(T_stock_options)

S_commodity = np.round(np.random.uniform(1000, 2000), 0)
r_commodity = np.round(np.random.uniform(0.03, 0.07), 3)
delta_commodity = np.round(np.random.uniform(0.01, 0.04), 3)
T_commodity_options = [0.25, 0.5, 0.75, 1.0]
T_commodity = np.random.choice(T_commodity_options)

S_USDCAD = np.round(np.random.uniform(1.20, 1.50), 2)
r_CAD = np.round(np.random.uniform(0.03, 0.07), 3)
r_USD = np.round(np.random.uniform(0.02, 0.06), 3)
T_fx_options = [0.25, 0.5, 0.75, 1.0]
T_fx = np.random.choice(T_fx_options)

S_stock2 = np.round(np.random.uniform(100, 200), 0)
r_stock2 = np.round(np.random.uniform(0.04, 0.08), 3)
q_stock2_actual = np.round(np.random.uniform(0.02, 0.05), 3)
T_stock2_options = [0.5, 0.75, 1.0, 1.5]
T_stock2 = np.random.choice(T_stock2_options)
F_stock2 = np.round(S_stock2 * np.exp((r_stock2 - q_stock2_actual) * T_stock2), 2)

S_commodity2 = np.round(np.random.uniform(1500, 2500), 0)
r_commodity2 = np.round(np.random.uniform(0.03, 0.07), 3)
delta_commodity2_actual = np.round(np.random.uniform(0.015, 0.04), 3)
T_commodity2_options = [0.5, 0.75, 1.0]
T_commodity2 = np.random.choice(T_commodity2_options)
F_commodity2 = np.round(S_commodity2 * np.exp((r_commodity2 - delta_commodity2_actual) * T_commodity2), 2)

print("="*70)
print("RÉSUMÉ DE LA GÉNÉRATION DES DONNÉES")
print("="*70)

print("\nProblème 1 - Prix à Terme d'une Action :")
print(f"  S₀ = ${S_stock:.0f}, r = {r_stock*100:.1f}%, q = {q_stock*100:.1f}%, T = {T_stock:.2f} années")

print("\nProblème 2 - Rendement de Dividende Implicite :")
print(f"  S₀ = ${S_stock2:.0f}, F = ${F_stock2:.2f}, r = {r_stock2*100:.1f}%, T = {T_stock2:.2f} années")

print("\nProblème 3 - Prix à Terme d'une Matière Première :")
print(f"  S₀ = ${S_commodity:.0f}, r = {r_commodity*100:.1f}%, δ = {delta_commodity*100:.1f}%, T = {T_commodity:.2f} années")

print("\nProblème 4 - Taux de Crédit-Bail Implicite :")
print(f"  S₀ = ${S_commodity2:.0f}, F = ${F_commodity2:.2f}, r = {r_commodity2*100:.1f}%, T = {T_commodity2:.2f} années")
print(f"  (δ réel utilisé pour générer F : {delta_commodity2_actual*100:.1f}%)")

print("\nProblème 5 & 6 - Contrats à Terme sur Devises :")
print(f"  S_USDCAD = {S_USDCAD:.2f}, r_CAD = {r_CAD*100:.1f}%, r_USD = {r_USD*100:.1f}%, T = {T_fx:.2f} années")

print("\n" + "="*70)
print("Données Générées avec Succès !")
print("="*70)

## 4. Solutions

### Problème 1 : Calculer le Prix à Terme d'une Action Versant des Dividendes

**Données :**
- Prix au comptant : $S_0 = \$100$
- Taux sans risque : $r = 5\%$ par an
- Rendement de dividende : $q = 2\%$ par an
- Échéance : $T = 1$ an

**Trouver :** Prix à terme $F_0^T$

In [ ]:
print("="*70)
print("PROBLÈME 1 : Prix à Terme d'une Action Versant des Dividendes")
print("="*70)

print(f"\nDonnées :")
print(f"  S₀ = ${S_stock:.2f}")
print(f"  r  = {r_stock*100:.1f}%")
print(f"  q  = {q_stock*100:.1f}%")
print(f"  T  = {T_stock:.1f} année")

print(f"\nFormule : F₀ᵀ = S₀ · e^((r-q)T)")

F_stock = S_stock * np.exp((r_stock - q_stock) * T_stock)

print(f"\nCalcul :")
print(f"  F₀ᵀ = {S_stock:.2f} × e^(({r_stock:.3f} - {q_stock:.3f}) × {T_stock:.1f})")
print(f"      = {S_stock:.2f} × e^({(r_stock - q_stock) * T_stock:.4f})")
print(f"      = {S_stock:.2f} × {np.exp((r_stock - q_stock) * T_stock):.6f}")
print(f"      = ${F_stock:.2f}")

print(f"\n" + "="*70)
print(f"RÉPONSE : Le prix à terme à 1 an est ${F_stock:.2f}")
print("="*70)

### Problème 2 : Calculer le Rendement de Dividende à partir du Prix à Terme

**Données :**
- Prix au comptant : $S_0 = \$150$
- Prix à terme : $F_0^T = \$156.82$
- Taux sans risque : $r = 6\%$ par an
- Échéance : $T = 1.5$ années

**Trouver :** Rendement de dividende implicite $q$

In [ ]:
print("="*70)
print("PROBLÈME 2 : Calculer le Rendement de Dividende à partir du Prix à Terme")
print("="*70)

print(f"\nDonnées :")
print(f"  S₀  = ${S_stock2:.2f}")
print(f"  F₀ᵀ = ${F_stock2:.2f}")
print(f"  r   = {r_stock2*100:.1f}%")
print(f"  T   = {T_stock2:.1f} années")

print(f"\nFormule : F₀ᵀ = S₀ · e^((r-q)T)")
print(f"\nRésolution pour q :")
print(f"  e^((r-q)T) = F₀ᵀ / S₀")
print(f"  (r-q)T = ln(F₀ᵀ / S₀)")
print(f"  r - q = ln(F₀ᵀ / S₀) / T")
print(f"  q = r - ln(F₀ᵀ / S₀) / T")

q_implied = r_stock2 - np.log(F_stock2 / S_stock2) / T_stock2

print(f"\nCalcul :")
print(f"  F₀ᵀ / S₀ = {F_stock2:.2f} / {S_stock2:.2f} = {F_stock2/S_stock2:.6f}")
print(f"  ln(F₀ᵀ / S₀) = ln({F_stock2/S_stock2:.6f}) = {np.log(F_stock2/S_stock2):.6f}")
print(f"  ln(F₀ᵀ / S₀) / T = {np.log(F_stock2/S_stock2):.6f} / {T_stock2:.1f} = {np.log(F_stock2/S_stock2)/T_stock2:.6f}")
print(f"  q = {r_stock2:.3f} - {np.log(F_stock2/S_stock2)/T_stock2:.6f}")
print(f"    = {q_implied:.6f}")
print(f"    = {q_implied*100:.2f}%")

print(f"\n" + "="*70)
print(f"RÉPONSE : Le rendement de dividende implicite est {q_implied*100:.2f}%")
print("="*70)

print(f"\nVérification :")
F_verify = S_stock2 * np.exp((r_stock2 - q_implied) * T_stock2)
print(f"  F₀ᵀ = {S_stock2:.2f} × e^(({r_stock2:.3f} - {q_implied:.6f}) × {T_stock2:.1f})")
print(f"      = ${F_verify:.2f} ✓")

### Problème 3 : Calculer le Prix à Terme d'une Matière Première

**Données :**
- Prix au comptant : $S_0 = \$1500$
- Taux sans risque : $r = 4\%$ par an
- Taux de crédit-bail (rendement de commodité) : $\delta = 1.5\%$ par an
- Échéance : $T = 0.5$ années (6 mois)

**Trouver :** Prix à terme $F_0^T$

In [ ]:
print("="*70)
print("PROBLÈME 3 : Prix à Terme d'une Matière Première")
print("="*70)

print(f"\nDonnées :")
print(f"  S₀ = ${S_commodity:.2f}")
print(f"  r  = {r_commodity*100:.1f}%")
print(f"  δ  = {delta_commodity*100:.1f}% (taux de crédit-bail/rendement de commodité)")
print(f"  T  = {T_commodity:.1f} années ({T_commodity*12:.0f} mois)")

print(f"\nFormule : F₀ᵀ = S₀ · e^((r-δ)T)")

F_commodity = S_commodity * np.exp((r_commodity - delta_commodity) * T_commodity)

print(f"\nCalcul :")
print(f"  F₀ᵀ = {S_commodity:.2f} × e^(({r_commodity:.3f} - {delta_commodity:.3f}) × {T_commodity:.1f})")
print(f"      = {S_commodity:.2f} × e^({(r_commodity - delta_commodity) * T_commodity:.5f})")
print(f"      = {S_commodity:.2f} × {np.exp((r_commodity - delta_commodity) * T_commodity):.6f}")
print(f"      = ${F_commodity:.2f}")

print(f"\n" + "="*70)
print(f"RÉPONSE : Le prix à terme à 6 mois est ${F_commodity:.2f}")
print("="*70)

### Problème 4 : Calculer le Taux de Crédit-Bail à partir du Prix à Terme

**Données :**
- Prix au comptant : $S_0 = \$2000$
- Prix à terme : $F_0^T = \$2045.30$
- Taux sans risque : $r = 5\%$ par an
- Échéance : $T = 1$ an

**Trouver :** Taux de crédit-bail implicite $\delta$

In [ ]:
print("="*70)
print("PROBLÈME 4 : Calculer le Taux de Crédit-Bail à partir du Prix à Terme")
print("="*70)

print(f"\nDonnées :")
print(f"  S₀  = ${S_commodity2:.2f}")
print(f"  F₀ᵀ = ${F_commodity2:.2f}")
print(f"  r   = {r_commodity2*100:.1f}%")
print(f"  T   = {T_commodity2:.1f} année")

print(f"\nFormule : F₀ᵀ = S₀ · e^((r-δ)T)")
print(f"\nRésolution pour δ :")
print(f"  e^((r-δ)T) = F₀ᵀ / S₀")
print(f"  (r-δ)T = ln(F₀ᵀ / S₀)")
print(f"  r - δ = ln(F₀ᵀ / S₀) / T")
print(f"  δ = r - ln(F₀ᵀ / S₀) / T")

delta_implied = r_commodity2 - np.log(F_commodity2 / S_commodity2) / T_commodity2

print(f"\nCalcul :")
print(f"  F₀ᵀ / S₀ = {F_commodity2:.2f} / {S_commodity2:.2f} = {F_commodity2/S_commodity2:.6f}")
print(f"  ln(F₀ᵀ / S₀) = ln({F_commodity2/S_commodity2:.6f}) = {np.log(F_commodity2/S_commodity2):.6f}")
print(f"  ln(F₀ᵀ / S₀) / T = {np.log(F_commodity2/S_commodity2):.6f} / {T_commodity2:.1f} = {np.log(F_commodity2/S_commodity2)/T_commodity2:.6f}")
print(f"  δ = {r_commodity2:.3f} - {np.log(F_commodity2/S_commodity2)/T_commodity2:.6f}")
print(f"    = {delta_implied:.6f}")
print(f"    = {delta_implied*100:.2f}%")

print(f"\n" + "="*70)
print(f"RÉPONSE : Le taux de crédit-bail implicite est {delta_implied*100:.2f}%")
print("="*70)

print(f"\nVérification :")
F_verify = S_commodity2 * np.exp((r_commodity2 - delta_implied) * T_commodity2)
print(f"  F₀ᵀ = {S_commodity2:.2f} × e^(({r_commodity2:.3f} - {delta_implied:.6f}) × {T_commodity2:.1f})")
print(f"      = ${F_verify:.2f} ✓")

### Problème 5 : Calculer les Prix à Terme des Devises (USDCAD et CADUSD)

**Données :**
- Taux de change au comptant USDCAD : $S_{USDCAD} = 1.35$ (1 USD = 1.35 CAD)
- Taux sans risque CAD : $r_{CAD} = 4.5\%$ par an
- Taux sans risque USD : $r_{USD} = 3.0\%$ par an
- Échéance : $T = 0.25$ années (3 mois)

**Trouver :** 
- Prix à terme $F_{USDCAD}$
- Prix à terme $F_{CADUSD}$

In [ ]:
print("="*70)
print("PROBLÈME 5 : Prix à Terme des Devises")
print("="*70)

print(f"\nDonnées :")
print(f"  S_USDCAD = {S_USDCAD:.4f} (prix de 1 USD en CAD)")
print(f"  r_CAD    = {r_CAD*100:.1f}%")
print(f"  r_USD    = {r_USD*100:.1f}%")
print(f"  T        = {T_fx:.2f} années ({T_fx*12:.0f} mois)")

print(f"\n" + "-"*70)
print("PARTIE A : Contrat à Terme USDCAD")
print("-"*70)

print(f"\nPour USDCAD (prix de USD en CAD) :")
print(f"  - Devise domestique : CAD (utiliser r_CAD)")
print(f"  - Devise étrangère : USD (utiliser r_USD)")
print(f"\nFormule : F_USDCAD = S_USDCAD · e^((r_CAD - r_USD)T)")

F_USDCAD = S_USDCAD * np.exp((r_CAD - r_USD) * T_fx)

print(f"\nCalcul :")
print(f"  F_USDCAD = {S_USDCAD:.4f} × e^(({r_CAD:.3f} - {r_USD:.3f}) × {T_fx:.2f})")
print(f"           = {S_USDCAD:.4f} × e^({(r_CAD - r_USD) * T_fx:.6f})")
print(f"           = {S_USDCAD:.4f} × {np.exp((r_CAD - r_USD) * T_fx):.6f}")
print(f"           = {F_USDCAD:.4f}")

print(f"\n" + "-"*70)
print("PARTIE B : Contrat à Terme CADUSD")
print("-"*70)

S_CADUSD = 1 / S_USDCAD

print(f"\nD'abord, calculer le taux de change au comptant CADUSD :")
print(f"  S_CADUSD = 1 / S_USDCAD = 1 / {S_USDCAD:.4f} = {S_CADUSD:.6f}")

print(f"\nPour CADUSD (prix de CAD en USD) :")
print(f"  - Devise domestique : USD (utiliser r_USD)")
print(f"  - Devise étrangère : CAD (utiliser r_CAD)")
print(f"\nFormule : F_CADUSD = S_CADUSD · e^((r_USD - r_CAD)T)")

F_CADUSD = S_CADUSD * np.exp((r_USD - r_CAD) * T_fx)

print(f"\nCalcul :")
print(f"  F_CADUSD = {S_CADUSD:.6f} × e^(({r_USD:.3f} - {r_CAD:.3f}) × {T_fx:.2f})")
print(f"           = {S_CADUSD:.6f} × e^({(r_USD - r_CAD) * T_fx:.6f})")
print(f"           = {S_CADUSD:.6f} × {np.exp((r_USD - r_CAD) * T_fx):.6f}")
print(f"           = {F_CADUSD:.6f}")

print(f"\n" + "="*70)
print(f"RÉPONSES :")
print(f"  F_USDCAD = {F_USDCAD:.4f}")
print(f"  F_CADUSD = {F_CADUSD:.6f}")
print("="*70)

### Problème 6 : Vérifier la Relation Réciproque

**Question :** 
1. Est-ce que $F_{USDCAD} = \frac{1}{F_{CADUSD}}$ ?
2. Est-ce que $F^P_{USDCAD} = \frac{1}{F^P_{CADUSD}}$ ?

In [ ]:
print("="*70)
print("PROBLÈME 6 : Relation Réciproque pour les Contrats à Terme et Contrats à Terme Prépayés")
print("="*70)

print(f"\n" + "-"*70)
print("PARTIE A : Prix à Terme Standards")
print("-"*70)

print(f"\nDu Problème 5 :")
print(f"  F_USDCAD = {F_USDCAD:.6f}")
print(f"  F_CADUSD = {F_CADUSD:.6f}")

print(f"\nVérifier si F_USDCAD = 1 / F_CADUSD :")
reciprocal_F = 1 / F_CADUSD
print(f"  1 / F_CADUSD = 1 / {F_CADUSD:.6f} = {reciprocal_F:.6f}")

difference_F = abs(F_USDCAD - reciprocal_F)
print(f"\n  F_USDCAD = {F_USDCAD:.6f}")
print(f"  1/F_CADUSD = {reciprocal_F:.6f}")
print(f"  Différence = {difference_F:.10f}")

if difference_F < 1e-6:
    print(f"\n  ✓ OUI ! F_USDCAD = 1/F_CADUSD (à la précision numérique près)")
else:
    print(f"\n  ✗ NON ! F_USDCAD ≠ 1/F_CADUSD")

print(f"\n" + "-"*70)
print("PARTIE B : Prix à Terme Prépayés")
print("-"*70)

print(f"\nCalculer les prix à terme prépayés :")
print(f"\nPour USDCAD (payer en CAD aujourd'hui, recevoir USD à l'échéance) :")
print(f"  Formule : F^P_USDCAD = S_USDCAD · e^(-r_USD · T)")
print(f"  (Actualiser au taux USD car vous achetez des USD)")

FP_USDCAD = S_USDCAD * np.exp(-r_USD * T_fx)

print(f"\n  F^P_USDCAD = {S_USDCAD:.4f} × e^(-{r_USD:.3f} × {T_fx:.2f})")
print(f"             = {S_USDCAD:.4f} × e^({-r_USD * T_fx:.6f})")
print(f"             = {S_USDCAD:.4f} × {np.exp(-r_USD * T_fx):.6f}")
print(f"             = {FP_USDCAD:.6f}")

print(f"\nPour CADUSD (payer en USD aujourd'hui, recevoir CAD à l'échéance) :")
print(f"  Formule : F^P_CADUSD = S_CADUSD · e^(-r_CAD · T)")
print(f"  (Actualiser au taux CAD car vous achetez des CAD)")

FP_CADUSD = S_CADUSD * np.exp(-r_CAD * T_fx)

print(f"\n  F^P_CADUSD = {S_CADUSD:.6f} × e^(-{r_CAD:.3f} × {T_fx:.2f})")
print(f"             = {S_CADUSD:.6f} × e^({-r_CAD * T_fx:.6f})")
print(f"             = {S_CADUSD:.6f} × {np.exp(-r_CAD * T_fx):.6f}")
print(f"             = {FP_CADUSD:.6f}")

print(f"\nVérifier si F^P_USDCAD = 1 / F^P_CADUSD :")
reciprocal_FP = 1 / FP_CADUSD
print(f"  1 / F^P_CADUSD = 1 / {FP_CADUSD:.6f} = {reciprocal_FP:.6f}")

difference_FP = abs(FP_USDCAD - reciprocal_FP)
print(f"\n  F^P_USDCAD   = {FP_USDCAD:.6f}")
print(f"  1/F^P_CADUSD = {reciprocal_FP:.6f}")
print(f"  Différence   = {difference_FP:.6f}")

if difference_FP < 1e-6:
    print(f"\n  ✓ OUI ! F^P_USDCAD = 1/F^P_CADUSD")
else:
    print(f"\n  ✗ NON ! F^P_USDCAD ≠ 1/F^P_CADUSD")

print(f"\n" + "="*70)
print("CONCLUSION :")
print("="*70)
print(f"\n1. Les contrats à terme standards SONT réciproques :")
print(f"   F_USDCAD = 1/F_CADUSD ✓")
print(f"\n2. Les contrats à terme prépayés ne sont PAS réciproques :")
print(f"   F^P_USDCAD ≠ 1/F^P_CADUSD ✗")
print(f"\n   Ceci est dû au fait que les contrats à terme prépayés utilisent des taux d'actualisation différents :")
print(f"   - F^P_USDCAD utilise r_USD = {r_USD*100:.1f}%")
print(f"   - F^P_CADUSD utilise r_CAD = {r_CAD*100:.1f}%")
print(f"\n   La relation réciproque ne s'applique que si r_USD = r_CAD.")
print("="*70)

## 5. Tableau Récapitulatif

In [ ]:
summary_data = {
    'Problème': [
        'Prix à Terme Action',
        'Rendement de Dividende Implicite',
        'Prix à Terme Matière Première',
        'Taux de Crédit-Bail Implicite',
        'Contrat à Terme USDCAD',
        'Contrat à Terme CADUSD',
        'Contrat à Terme Prépayé USDCAD',
        'Contrat à Terme Prépayé CADUSD'
    ],
    'Résultat': [
        f'${F_stock:.2f}',
        f'{q_implied*100:.2f}%',
        f'${F_commodity:.2f}',
        f'{delta_implied*100:.2f}%',
        f'{F_USDCAD:.4f}',
        f'{F_CADUSD:.6f}',
        f'{FP_USDCAD:.6f}',
        f'{FP_CADUSD:.6f}'
    ]
}

df_summary = pd.DataFrame(summary_data)

print("\n" + "="*70)
print("RÉSUMÉ DE TOUS LES RÉSULTATS")
print("="*70)
print(df_summary.to_string(index=False))

print("\n" + "="*70)
print("POINTS CLÉS")
print("="*70)
print("\n1. La tarification à terme dépend du coût de portage :")
print("   - Actions : F = S·e^((r-q)T)")
print("   - Matières premières : F = S·e^((r-δ)T)")
print("   - Devises : F = S·e^((r_d-r_f)T)")

print("\n2. Les contrats à terme prépayés actualisent le prix à terme :")
print("   - F^P = F·e^(-rT)")
print("   - Pour les devises, utiliser le taux étranger")

print("\n3. Relation réciproque pour les contrats à terme sur devises :")
print("   - Contrats à terme standards : F_USDCAD = 1/F_CADUSD ✓")
print("   - Contrats à terme prépayés : F^P_USDCAD ≠ 1/F^P_CADUSD ✗")
print("     (sauf si r_USD = r_CAD)")
print("="*70)